In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("ggplot")

import torch
import torch.nn as nn
import torchvision.transforms as transforms

import timm

import gc
import os
import time
import random
from datetime import datetime

from PIL import Image
from tqdm.notebook import tqdm
from sklearn import model_selection, metrics

In [ ]:
def seed_everything(seed):
    """
    Seeds basic parameters for reproductibility of results
    
    Arguments:
        seed {int} -- Number of the seed
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(42)

In [ ]:
# general global variables
DATA_PATH = "/kaggle/input/cassava-leaf-disease-classification"
TRAIN_PATH = "/kaggle/input/cassava-leaf-disease-classification/train_images"
TRAIN_PATH = "/kaggle/input/cassava-leaf-disease-classification/test_images/"
MODEL_PATH = (
    "/kaggle/input/vit-base-models-pretrained-pytorch/jx_vit_base_p16_224-80ecf9dd.pth"
)
BEST_MODEL = "/kaggle/working/best_model.pth"
SUBMISSION_FILE = "submission.csv"

# model specific global variables
IMG_SIZE = 224
BATCH_SIZE = 16
LR = 2e-05
GAMMA = 0.7
N_EPOCHS = 10

In [ ]:
df = pd.read_csv(os.path.join(DATA_PATH, "train.csv"))
df.head()

In [ ]:
df.info()

In [ ]:
df.label.value_counts().plot(kind="bar")

In [ ]:
train_df, valid_df = model_selection.train_test_split(
    df, test_size=0.1, random_state=42, stratify=df.label.values
)

In [ ]:
class CassavaDataset(torch.utils.data.Dataset):
    """
    Helper Class to create the pytorch dataset
    """

    def __init__(self, df, data_path=DATA_PATH, mode="train", transforms=None):
        super().__init__()
        self.df_data = df.values
        self.data_path = data_path
        self.transforms = transforms
        self.mode = mode
        self.data_dir = "train_images" if mode == "train" else "test_images"

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, index):
        img_name, label = self.df_data[index]
        img_path = os.path.join(self.data_path, self.data_dir, img_name)
        img = Image.open(img_path).convert("RGB")

        if self.transforms is not None:
            image = self.transforms(img)

        return image, label

In [ ]:
# create image augmentations
transforms_train = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomVerticalFlip(p=0.3),
        transforms.RandomResizedCrop(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

transforms_valid = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

In [ ]:
print("Available Vision Transformer Models: ")
timm.list_models("vit*")

In [ ]:
from torch.utils.tensorboard import SummaryWriter
class ViTBase16(nn.Module):
    def __init__(self, n_classes, pretrained=False):

        super(ViTBase16, self).__init__()

        self.model = timm.create_model("vit_base_patch16_224", pretrained=False)
        if pretrained:
            self.model.load_state_dict(torch.load(MODEL_PATH))

        self.model.head = nn.Linear(self.model.head.in_features, n_classes)

    def forward(self, x):
        x = self.model(x)
        return x

    def train_one_epoch(self, train_loader, criterion, optimizer, device, writer, epoch):
        # keep track of training loss
        epoch_loss = 0.0
        epoch_accuracy = 0.0

        ###################
        # train the model #
        ###################
        self.model.train()
        for i, (data, target) in enumerate(train_loader):
            # move tensors to GPU if CUDA is available
            if device.type == "cuda":
                data, target = data.cuda(), target.cuda()

            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = self.forward(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # Calculate Accuracy
            accuracy = (output.argmax(dim=1) == target).float().mean()
            # update training loss and accuracy
            epoch_loss += loss
            epoch_accuracy += accuracy
            if i % 20 == 0:
                print(f"BATCH {i+1}/{len(train_loader)} - LOSS: {loss.item():.4f} - ACCURACY: {accuracy.item():.4f}")
                writer.add_scalar('Training Loss', loss.item(), epoch * len(train_loader) + i)
                writer.add_scalar('Training Accuracy', accuracy.item(), epoch * len(train_loader) + i)
            optimizer.step()

        return epoch_loss / len(train_loader), epoch_accuracy / len(train_loader)

    def valid_one_epoch(self, valid_loader, criterion, device, writer, epoch):
        # keep track of validation loss
        valid_loss = 0.0
        valid_accuracy = 0.0

        ######################
        # validate the model #
        ######################
        self.model.eval()
        for data, target in valid_loader:
            # move tensors to GPU if CUDA is available
            if device.type == "cuda":
                data, target = data.cuda(), target.cuda()
            elif device.type == "xla":
                data = data.to(device, dtype=torch.float32)
                target = target.to(device, dtype=torch.int64)

            with torch.no_grad():
                # forward pass: compute predicted outputs by passing inputs to the model
                output = self.model(data)
                # calculate the batch loss
                loss = criterion(output, target)
                # Calculate Accuracy
                accuracy = (output.argmax(dim=1) == target).float().mean()
                # update average validation loss and accuracy
                valid_loss += loss
                valid_accuracy += accuracy

        writer.add_scalar('Validation Loss', valid_loss / len(valid_loader), epoch)
        writer.add_scalar('Validation Accuracy', valid_accuracy / len(valid_loader), epoch)

        return valid_loss / len(valid_loader), valid_accuracy / len(valid_loader)

In [ ]:
def fit_gpu(model, epochs, device, criterion, optimizer, train_loader, valid_loader=None):
    writer = SummaryWriter()
    valid_loss_min = np.Inf
    train_losses = []
    valid_losses = []
    train_accs = []
    valid_accs = []

    for epoch in  range(1,epochs+1):  #调用数据和模型进行训练-Log
        gc.collect()  #通过gc清理内存
        print(f"{'='*50}")
        print(f"EPOCH{epoch}-TRAINING...")

        train_loss,train_acc=model.train_one_epoch(train_loader,criterion,optimizer,device,writer,epoch)
        print(f"\n\t[TRAIN] EPOCH{epoch}-LOSS:{train_loss},ACCURACY:{train_acc}\n")
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        gc.collect()

        #valid
        if valid_loader is not None:
            gc.collect()
            print(f"EPOCH{epoch}-VALIDATING...")
            valid_loss,valid_acc = model.valid_one_epoch(valid_loader,criterion,device,writer,epoch)
            print(f"\t[VALID] LOSS:{valid_loss},ACCURACY:{valid_acc}\n")
            valid_losses.append(valid_loss)
            valid_accs.append(valid_acc)
            gc.collect()
            #save !!!
            if valid_loss<=valid_loss_min and epoch!=1:
                print("Validation loss decreased ({:.4f} -->{:.4f}). Saving model...".format(valid_loss_min,valid_loss))
                torch.save(model.state_dict(),'best_model.pth')
                valid_loss_min=valid_loss
    writer.close()
    return {
        "train_loss":train_losses,
        "valid_losses":valid_losses,
        "train_acc":train_accs,
        "valid_acces":valid_accs,   
    }

In [ ]:
def run():
    df = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
    train_df, valid_df = model_selection.train_test_split(df, test_size=0.1, random_state=42, shuffle=True,
                                                         stratify=df.label.values)

    train_dataset = CassavaDataset(train_df, transforms=transforms_train)
    valid_dataset = CassavaDataset(valid_df, transforms=transforms_valid)


    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        drop_last=True,
        num_workers=4,
    )
    valid_loader = torch.utils.data.DataLoader(
        dataset=valid_dataset,
        batch_size=BATCH_SIZE,
        drop_last=True,
        num_workers=4,
    )

    criterion = nn.CrossEntropyLoss()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    lr = LR
    model = ViTBase16(n_classes=5, pretrained=True)
    model=model.cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    #开始训练前
    start_time=datetime.now()
    logs = fit_gpu(model=model,
                       epochs=N_EPOCHS,
                       device=device,
                       criterion=criterion,
                       optimizer=optimizer,
                       train_loader=train_loader,
                       valid_loader=valid_loader)

    print(f"Execution time:{datetime.now() - start_time}")
    torch.save(model.state_dict(),f'end_model.pth')

In [ ]:
# torch.set_default_tensor_type("torch.FloatTensor")
# a = run()

In [ ]:
BEST_MODEL="/kaggle/input/cassava-vit-b16/best_model.pth"
test_df = pd.read_csv("/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv")
transforms_test = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

test_dataset = CassavaDataset(test_df, transforms=transforms_test, mode="test")

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pretrained_model = ViTBase16(n_classes=5)
pretrained_model.load_state_dict(torch.load(BEST_MODEL, map_location=device))
pretrained_model =pretrained_model.to(device)
pretrained_model.eval()

predictions = []

with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = pretrained_model(images)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())
        
test_df['label'] = predictions

# Save to CSV
test_df.to_csv(SUBMISSION_FILE, index=False)
print(f"Submission file saved to {SUBMISSION_FILE}")